# Generalized representer formula

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from Kernels import get_gaussianRBF
from KernelTools import *
from EquationModel import CholInducedRKHS

## Introduction to `InducedRKHS`

Given

- Kernel: $K: \mathbb{R}^d \times \mathbb{R}^d \to \mathbb{R}$,
- Basis points: $X = \left\{x_1,\dots,x_M\right\} \subset \mathbb{R}^d$,
- Basis operators: $\phi = \left[\phi_1,\dots,\phi_K \right]$,

we can represent an element of the correponding RKHS $H_K$ of $K$ as

$$
\begin{align*}
u(x) &= \sum_{m=1}^M \phi_1 \left(K(x,x')\right)_{x' = x_m} \alpha_{1,m} + \dots + \sum_{m=1}^M \phi_K \left(K(x,x')\right)_{x' = x_m} \alpha_{K,m} \\
&:= \sum_{m=1}^M  K(x,\phi_{1,m}) \alpha_{1,m} + \dots + \sum_{m=1}^M K(x,\phi_{K,m}) \alpha_{K,m} \\
&:= K(x,\phi_{1})^\top \alpha_{1} + \dots + K(x,\phi_{K})^\top \alpha_{K} \\
&:= K(x,\phi)^\top \alpha \\
\end{align*} 
$$
where $\{\alpha_1,\dots,\alpha_K\} \subset \mathbb{R}^M$ and $\alpha = [\alpha_1,\dots,\alpha_K] \in \mathbb{R}^{MK}$.

In [2]:
# Given
K = get_gaussianRBF(0.1)
basis_pts = jax.random.normal(jax.random.PRNGKey(0), shape = (100,2))
basis_ops = (eval_k, dx_k, dxx_k)

We can use the class `CholInducedRKHS` to create the function $u$

In [3]:
# Create an object
u_model = CholInducedRKHS(
    basis_pts,
    basis_ops,
    K 
    )

If we know the value of  $\alpha$

In [4]:
# alpha 
alpha = jnp.ones(len(basis_pts)*len(basis_ops))

In order to evaluate $u$, we call the method `point_evaluate`

In [ ]:
# Evaluation pts
eval_pts = jax.random.normal(jax.random.PRNGKey(0), shape = (500,2))
# Evaluate u at eval_pts
u_model.point_evaluate(eval_pts,alpha)

## Kernel ridge regression

Usually, when we are doing kernel methods we want to solve the problem 

$$
\underset{u \in H_K}{\min} \|u\|_{H_K}^2 \quad \text{s.t.} \quad u(x_i) = y_i
$$

where $X = \{(x_i,y_i)\}_{i=1}^N \subset \mathbb{R}^d$. Whose solution is given by 

$$
u(x) = K(x,X) K(x,X)^{-1} y
$$